In [5]:
!pip install git+https://github.com/kvoyager/GmdhPy.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/kvoyager/GmdhPy.git to /tmp/pip-req-build-lprjkjhk
  Running command git clone -q https://github.com/kvoyager/GmdhPy.git /tmp/pip-req-build-lprjkjhk
  Created wheel for GmdhPy: filename=GmdhPy-2.0-py2.py3-none-any.whl size=19299 sha256=a7bb101ee61dd71bd51171230c64493c5ec480b94ce21b72f4c89e58ab055161
  Stored in directory: /tmp/pip-ephem-wheel-cache-0njll6m0/wheels/0d/1b/b1/3b0ed54539b36d869f01f28ba4be6b0cc585254b913fc61e7b
Successfully built GmdhPy


In [28]:
!pip install heamy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for heamy: filename=heamy-0.0.7-py2.py3-none-any.whl size=15366 sha256=c262ba85351876e5fc6c9e78f7a20f7eece4c873f666a4799c063a0e42b01f78
  Stored in directory: /root/.cache/pip/wheels/f5/6c/da/55718ad26a9c8d3528b50edc2676fb33b0fb2e2b04e54cc882
Successfully built heamy


In [34]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, balanced_accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score,  roc_auc_score
from sklearn.metrics import confusion_matrix
from heamy.estimator import Classifier
from heamy.pipeline import ModelsPipeline
from heamy.dataset import Dataset
from warnings import simplefilter

simplefilter('ignore')
%matplotlib inline 

In [9]:
from google.colab import files
uploaded = files.upload()

Saving Fraud_cut.csv to Fraud_cut.csv


In [10]:
orig_df = pd.read_csv('./Fraud_cut.csv', sep=',')
orig_df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,688,CASH_IN,23557.12,C867750533,8059.00,31616.12,C1026934669,169508.66,145951.53,0,0
1,274,PAYMENT,6236.13,C601099070,0.00,0.00,M701283411,0.00,0.00,0,0
2,133,PAYMENT,33981.87,C279540931,18745.72,0.00,M577905776,0.00,0.00,0,0
3,225,CASH_OUT,263006.42,C11675531,20072.00,0.00,C529577791,390253.56,653259.98,0,0
4,249,CASH_OUT,152013.74,C530649214,20765.00,0.00,C1304175579,252719.19,404732.93,0,0


In [11]:
df = orig_df.dropna()
df.isnull().sum()

step              0
type              0
amount            0
nameOrig          0
oldbalanceOrg     0
newbalanceOrig    0
nameDest          0
oldbalanceDest    0
newbalanceDest    0
isFraud           0
isFlaggedFraud    0
dtype: int64

В датасете нет пропусков данных

Удалим из датасета столбцы с именами отправителей и получателей переводов

In [12]:
df = df.drop(['nameOrig', 'nameDest'], axis=1)

Выполняем кодирование категориальных признаков

In [15]:
le = LabelEncoder()
df["type"] = le.fit_transform(df[['type']])
df.dtypes

step                int64
type                int64
amount            float64
oldbalanceOrg     float64
newbalanceOrig    float64
oldbalanceDest    float64
newbalanceDest    float64
isFraud             int64
isFlaggedFraud      int64
dtype: object

**Формирование обучающей и тестовой выборок**

In [16]:
y = df['isFraud']
x = df.drop('isFraud', axis=1)

In [17]:
y.value_counts()

0    4491
1       9
Name: isFraud, dtype: int64

В целевом признаке присутствует критичный дисбаланас классов. Это может сказаться на качетсве обучения модели. Обученная модель с большой вероятностью будет игнорировать меньший класс (isFraud = 1), который в нашем случае представляет наибольший интерес. Для решения этой проблемы воспользуемся алгоритмом SMOTE. 

In [18]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
over = SMOTE(sampling_strategy=0.1)
under = RandomUnderSampler(sampling_strategy=0.5)
steps = [('o', over), ('u', under)]
pipeline = Pipeline(steps=steps)
x_balanced, y_balanced = pipeline.fit_resample(x, y)

In [19]:
y_balanced.value_counts()

0    898
1    449
Name: isFraud, dtype: int64

In [20]:
x_train, x_test, y_train, y_test = train_test_split(x_balanced, y_balanced, test_size=0.3, random_state=2)

**Обучение ансамблевых моделей**

In [48]:
#функция, выводящая метрики
def print_metrics(y_test, y_pred):
  print(f"Balanced Accuracy: {balanced_accuracy_score(y_test, y_pred)}")
  print(f"Precision: {precision_score(y_test, y_pred, average='weighted', labels=np.unique(y_pred))}")
  print(f"Recall: {recall_score(y_test, y_pred, average='weighted', labels=np.unique(y_pred))}")
  print(f"F1: {f1_score(y_test, y_pred, average='weighted', labels=np.unique(y_pred))}")

Случайный лес

In [49]:
model1 = RandomForestClassifier()
est1 = model1.fit(x_train, y_train)
y_pred1 = est1.predict(x_test)
print_metrics(y_test, y_pred1)

Balanced Accuracy: 0.99812734082397
Precision: 0.9975486277644551
Recall: 0.9975308641975309
F1: 0.9975330048588832


Градиентный бустинг

In [50]:
model2 = GradientBoostingClassifier()
est2 = model2.fit(x_train, y_train)
y_pred2 = est2.predict(x_test)
print_metrics(y_test, y_pred2)

Balanced Accuracy: 0.99812734082397
Precision: 0.9975486277644551
Recall: 0.9975308641975309
F1: 0.9975330048588832


Стекинг

In [31]:
dataset = Dataset(x_train, y_train, x_test)

# модели первого уровня
model_tree = Classifier(dataset=dataset, estimator=DecisionTreeClassifier, name='tree')
model_rf = Classifier(dataset=dataset, estimator=RandomForestClassifier, parameters={'n_estimators': 50},name='rf')

In [37]:
# Первый уровень - две модели: дерево и лес
# Второй уровень: логическая регрессия

pipeline = ModelsPipeline(model_tree, model_rf)
stack_ds = pipeline.stack(k=10, seed=1)
# модель второго уровня
model3 = Classifier(dataset=stack_ds, estimator=LogisticRegression)
results = model3.validate(k=10, scorer= roc_auc_score)


Metric: roc_auc_score
Folds accuracy: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9994588744588744, 0.9995098039215686, 1.0]
Mean accuracy: 0.9998968678380443
Standard Deviation: 0.0002065784640932893
Variance: 4.267466182714242e-08


Попробуем подобать гиперпараметры для моделей леса и бустинга

In [51]:
f_params = {'n_estimators': [50, 100, 120, 150, 170],  'criterion': ['gini', 'entropy'], 'min_samples_leaf': [ 2, 3, 4]}
gs1 = GridSearchCV(model1, f_params,
                  cv=ShuffleSplit(n_splits=8), scoring='roc_auc',
                  return_train_score=True, n_jobs=-1)
gs1.fit(x_balanced, y_balanced)

GridSearchCV(cv=ShuffleSplit(n_splits=8, random_state=None, test_size=None, train_size=None),
             estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'min_samples_leaf': [2, 3, 4],
                         'n_estimators': [50, 100, 120, 150, 170]},
             return_train_score=True, scoring='roc_auc')

In [53]:
est3 = gs1.best_estimator_.fit(x_train, y_train)
y_pred3 = est3.predict(x_test)
print_metrics(y_test, y_pred3)

Balanced Accuracy: 0.99812734082397
Precision: 0.9975486277644551
Recall: 0.9975308641975309
F1: 0.9975330048588832


In [47]:
param_range = np.arange(10, 300, 20)
gb_params = {'n_estimators': param_range,  'subsample': [0.0, 0.5, 1.0], 'loss': [ 'log_loss', 'exponential']}
gs2 = GridSearchCV(model2, gb_params, cv=8, n_jobs=-1, scoring='roc_auc')
gs2.fit(x_balanced, y_balanced)

GridSearchCV(cv=8, estimator=GradientBoostingClassifier(), n_jobs=-1,
             param_grid={'loss': ['log_loss', 'exponential'],
                         'n_estimators': array([ 10,  30,  50,  70,  90, 110, 130, 150, 170, 190, 210, 230, 250,
       270, 290]),
                         'subsample': [0.0, 0.5, 1.0]},
             scoring='roc_auc')

In [52]:
est4 = gs2.best_estimator_.fit(x_train, y_train)
y_pred4 = est4.predict(x_test)
print_metrics(y_test, y_pred4)

Balanced Accuracy: 0.9943820224719101
Precision: 0.992750197005516
Recall: 0.9925925925925926
F1: 0.9926114924839182


**Модель многослойного персиптрона**

In [54]:
from sklearn.neural_network import MLPClassifier

In [55]:
model4 = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)
est5 = model4.fit(x_train, y_train)
y_pred5 = est5.predict(x_test)
print_metrics(y_test, y_pred5)

Balanced Accuracy: 0.5730337078651685
Precision: 0.7877352762598665
Recall: 0.43703703703703706
F1: 0.35464259777985263


In [59]:
mlp_params = {'hidden_layer_sizes' : [(7, 3), (9, 4), (11, 5)],  'solver': ['lbfgs', 'sgd', 'adam'], 'alpha': [ 1e-5, 1e-4, 1e-3, 1e-2, 1e-1]}
gs3 = GridSearchCV(model4, mlp_params,
                  cv=ShuffleSplit(n_splits=7), scoring='roc_auc',
                  return_train_score=True, n_jobs=-1)
gs3.fit(x_balanced, y_balanced)

GridSearchCV(cv=ShuffleSplit(n_splits=7, random_state=None, test_size=None, train_size=None),
             estimator=MLPClassifier(alpha=1e-05, hidden_layer_sizes=(5, 2),
                                     random_state=1, solver='lbfgs'),
             n_jobs=-1,
             param_grid={'alpha': [1e-05, 0.0001, 0.001, 0.01, 0.1],
                         'hidden_layer_sizes': [(7, 3), (9, 4), (11, 5)],
                         'solver': ['lbfgs', 'sgd', 'adam']},
             return_train_score=True, scoring='roc_auc')

In [60]:
gs3.best_params_

{'alpha': 1e-05, 'hidden_layer_sizes': (9, 4), 'solver': 'adam'}

In [61]:
est6 = gs3.best_estimator_.fit(x_train, y_train)
y_pred6 = est6.predict(x_test)
print_metrics(y_test, y_pred6)

Balanced Accuracy: 0.951921511154535
Precision: 0.9505342328486205
Recall: 0.9481481481481482
F1: 0.9485965375906149


In [80]:
scaler = StandardScaler().fit(x_train)
x_train_scaled = pd.DataFrame(scaler.transform(x_train), columns=x_train.columns)
x_test_scaled = pd.DataFrame(scaler.transform(x_test), columns=x_test.columns)

In [73]:
x_test_scaled.shape

(405, 8)

In [77]:
y_test.shape

(405,)

In [81]:
from gmdhpy.gmdh import Classifier
model5 = Classifier(n_jobs=-1)
model5.fit(np.array(x_train_scaled), np.array(y_train))
model5.predict(np.array(x_test_scaled))
#print_metrics(y_test, y_pred7)

train layer0 in 0.27 sec
train layer1 in 1.31 sec
train layer2 in 1.35 sec
train layer3 in 1.43 sec
train layer4 in 1.31 sec
train layer5 in 1.31 sec
train layer6 in 1.35 sec
train layer7 in 1.27 sec
train layer8 in 0.80 sec
train layer9 in 0.73 sec
train layer10 in 0.72 sec


ValueError: ignored